In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
# Simplified action space (digits 0-9 and signs)
actions = ['0','1','2','3','4','5','6','7','8','9', '+', '-']
num_actions = len(actions)

# Environment with one-hot operator encoding
def generate_problem():
    a = np.random.randint(0, 10)
    b = np.random.randint(0, 10)
    op = np.random.choice(['+', '-'])
    correct = a + b if op == '+' else a - b
    # State: [a/10, b/10, is_addition, is_subtraction]
    state = np.array([a/10.0, b/10.0, 1.0 if op == '+' else 0.0, 0.0 if op == '+' else 1.0])
    return state, correct

In [ ]:
model = tf.keras.Sequential([
    layers.Dense(8, activation='relu', input_shape=(4,)),
    layers.Dense(8, activation='relu'),
    layers.Dense(1, activation='relu')
])
model.compile(optimizer="adam", loss="mse")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Batched training loop
def train(num_episodes=1000):
    state, correct = zip(*[generate_problem() for x in range(num_episodes)])
    state = np.array(state, dtype=np.float32)  # Shape: (1000, 3)
    correct = np.array(correct, dtype=np.float32).reshape(-1, 1)
    model.fit(state, correct, epochs=500, verbose=0)

train()

In [ ]:
import numpy as np

def test(num_problems=100):
    correct_count = 0

    # Generate test problems
    state, correct = zip(*[generate_problem() for _ in range(num_problems)])

    # Convert state to NumPy array for model input
    state = np.array(state, dtype=np.float32)

    # Predict answers (already a NumPy array)
    predicted_answer = model.predict(state).tolist()  # Convert to Python list

    # Compare predicted vs. actual answers
    for i in range(len(predicted_answer)):
        if round(predicted_answer[i][0]) == int(correct[i]):  # Ensure correct[i] is an integer
            correct_count += 1

    # Calculate accuracy
    accuracy = (correct_count / num_problems) * 100
    print(f"Accuracy: {accuracy:.2f}%")

# Run the test
test()


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
Accuracy: 79.00%
